In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import yagmail

In [55]:
def classmate_emails(cov_id):

    df = pd.read_csv("sample_student_data.csv")
    df = df.set_index("id_number")
    classes_list = df.loc[int(cov_id), "classes"].split(",")

    df['classmates'] = df.apply(lambda row: True if any([item in row['classes'] for item in classes_list]) else False, axis = 1) 
    df['classmates'] = df['classmates']*df['obf_email']
    df = df.replace(r'', np.nan, regex=True)
    df = df.dropna()
    df = df[["name","obf_email"]]
    
    filename = f"{cov_id}_classmates_list.csv"
    df.to_csv(filename)
    
    return df


classmate_emails("202869")

,name,obf_email
id_number,,
202869,Sabrina Sy,sabrina.sy@obf.ateneo.edu
204995,Martine Uysipuo,martine.uysipuo@obf.ateneo.edu
203897,Raelyn Go,raelyn.go@obf.ateneo.edu
200594,Zachariah Tate,zachariah.tate@obf.ateneo.edu
201168,Justus Mckay,justus.mckay@obf.ateneo.edu
...,...,...
207156,Kylan Stevenson,kylan.stevenson@obf.ateneo.edu
202511,Dania Stuart,dania.stuart@obf.ateneo.edu
208067,Cruz Skinner,cruz.skinner@obf.ateneo.edu


In [72]:
# https://mathigon.org/course/programming-in-python/project-2-mail-merge

def message(row):
    return f"""
    Dear {row.name},

    One of your classmates has notified us that they have been experiencing symptoms of COVID-19. If you are experiencing any symptoms, please let us know immediately through our Telegram bot (ADMUCovid_bot). You may also contact the bot to for other pertinent information:

    You may use the following commands:
        /ihavesymptoms - When you are experiencing common symptoms
        /symptoms - Shows a list of symptoms of covid
        /contacts - Shows a list of important contact information
        /class status - Shows a status update of whether or not a class had a report of covid
        /statusupdate - Shows the current COVID data of the country
        /keepsafe - Ways to keep  safe in covid 

    Sincerely,
    ADMU COVID-19 Contact Tracing Team
    """

def send_email():
    yagmail.register('admucontacttracing@gmail.com','ADMUCT2021')
    yag = yagmail.SMTP('admucontacttracing@gmail.com')
    recipient_details = pd.read_csv(f"{cov_id}_classmates_list.csv")

    for row in recipient_details.itertuples():
        yag.send(to=row.obf_email,
                 subject="ADMU Contact Tracing",
                 contents = message(row)